In [35]:
products = [
    {
        "product_name": "떡케익5호",
        "quantity": 1,
        "price": 54000
    },
    {
        "product_name": "무지개 백설기 케익",
        "quantity": 1,
        "price": 51500
    },
    {
        "product_name": "미니 백설기",
        "quantity": 35,
        "price": 31500
    },
    {
        "product_name": "개별 모듬팩",
        "quantity": 1,
        "price": 13500
    }
]
products

[{'product_name': '떡케익5호', 'quantity': 1, 'price': 54000},
 {'product_name': '무지개 백설기 케익', 'quantity': 1, 'price': 51500},
 {'product_name': '미니 백설기', 'quantity': 35, 'price': 31500},
 {'product_name': '개별 모듬팩', 'quantity': 1, 'price': 13500}]

In [38]:
print(type(products))
print(type(products[0]))

<class 'list'>
<class 'dict'>


In [1]:
queried_result = [
    {
        "id": 10,
        "created_at": "2024-05-17T12:00:00Z",
        "order_status": "주문 완료",
        "items": [
            {
                "product_name": "떡케익5호",
                "quantity": 1,
                "price": 54000.0
            },
            {
                "product_name": "무지개 백설기 케익",
                "quantity": 1,
                "price": 51500.0
            }
        ]
    },
]
queried_result

[{'id': 10,
  'created_at': '2024-05-17T12:00:00Z',
  'order_status': '주문 완료',
  'items': [{'product_name': '떡케익5호', 'quantity': 1, 'price': 54000.0},
   {'product_name': '무지개 백설기 케익', 'quantity': 1, 'price': 51500.0}]}]

In [2]:
import json

queried_result = [
    {
        "id": 10,
        "created_at": "2024-05-17T12:00:00Z",
        "order_status": "주문 완료",
        "items": [
            {
                "product_name": "떡케익5호",
                "quantity": 1,
                "price": 54000.0
            },
            {
                "product_name": "무지개 백설기 케익",
                "quantity": 1,
                "price": 51500.0
            }
        ]
    },
]
queried_result

print(json.dumps(queried_result, indent=4, ensure_ascii=False))

[
    {
        "id": 10,
        "created_at": "2024-05-17T12:00:00Z",
        "order_status": "주문 완료",
        "items": [
            {
                "product_name": "떡케익5호",
                "quantity": 1,
                "price": 54000.0
            },
            {
                "product_name": "무지개 백설기 케익",
                "quantity": 1,
                "price": 51500.0
            }
        ]
    }
]


## 주문 변경 처리 체인

In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

order_change_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a customer service assistant responsible for processing order changes based on customer input and existing order details.
            
            Given the 'queried_result' which contains existing order details and the 'input' which contains the customer's desired order changes, combine these to finalize the order changes.
            
            Follow these steps:
            1. Review the 'queried_result' to understand the original order details.
            2. Review the 'input' to understand the customer's desired changes.
            3. Combine both sets of information to produce the updated order details.
            
            Ensure that the final output clearly lists the original items and the updated items in the order. If there is any mismatch or inconsistency, highlight it appropriately.
            
            """
        ),
        ("human", "input:{input}\nqueried_result:{queried_result}")
    ]
)

order_change_prompt


ChatPromptTemplate(input_variables=['input', 'queried_result'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a customer service assistant responsible for processing order changes based on customer input and existing order details.\n            \n            Given the 'queried_result' which contains existing order details and the 'input' which contains the customer's desired order changes, combine these to finalize the order changes.\n            \n            Follow these steps:\n            1. Review the 'queried_result' to understand the original order details.\n            2. Review the 'input' to understand the customer's desired changes.\n            3. Combine both sets of information to produce the updated order details.\n            \n            Ensure that the final output clearly lists the original items and the updated items in the order. If there is any mismatch or inconsistency, highlight it appropriately.

In [4]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [5]:
order_change_chain = order_change_prompt | model

In [6]:
response = order_change_chain.invoke(
    {"input": "무지개 백설기 케익만 주문할게요",
     "queried_result": queried_result}
    )
response

AIMessage(content='Original Order Details:\n- Order ID: 10\n- Created At: 2024-05-17T12:00:00Z\n- Order Status: 주문 완료\n- Items:\n    1. Product Name: 떡케익5호, Quantity: 1, Price: 54000.0\n    2. Product Name: 무지개 백설기 케익, Quantity: 1, Price: 51500.0\n\nRequested Order Changes:\n- New Item: Product Name: 무지개 백설기 케익, Quantity: 1\n\nUpdated Order Details:\n- Order ID: 10\n- Created At: 2024-05-17T12:00:00Z\n- Order Status: 주문 완료\n- Items:\n    1. Product Name: 떡케익5호, Quantity: 1, Price: 54000.0\n    2. Product Name: 무지개 백설기 케익, Quantity: 1, Price: 51500.0 (No change)\n\nIt seems that the requested change is already included in the original order. No further action is needed.', response_metadata={'token_usage': {'completion_tokens': 272, 'prompt_tokens': 294, 'total_tokens': 566}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-439bbeb0-f62d-40aa-9981-2a8d3b562ca2-0')

In [7]:
print(response.content)

Original Order Details:
- Order ID: 10
- Created At: 2024-05-17T12:00:00Z
- Order Status: 주문 완료
- Items:
    1. Product Name: 떡케익5호, Quantity: 1, Price: 54000.0
    2. Product Name: 무지개 백설기 케익, Quantity: 1, Price: 51500.0

Requested Order Changes:
- New Item: Product Name: 무지개 백설기 케익, Quantity: 1

Updated Order Details:
- Order ID: 10
- Created At: 2024-05-17T12:00:00Z
- Order Status: 주문 완료
- Items:
    1. Product Name: 떡케익5호, Quantity: 1, Price: 54000.0
    2. Product Name: 무지개 백설기 케익, Quantity: 1, Price: 51500.0 (No change)

It seems that the requested change is already included in the original order. No further action is needed.


In [19]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import OpenAI

model = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.0)

# Define the desired data structure for order details.
class OrderItem(BaseModel):
    product_name: str = Field(description="The name of the product")
    quantity: int = Field(description="The quantity of the product")
    price: float = Field(description="The price of the product")

class OrderDetails(BaseModel):
    id: int = Field(description="The order ID")
    created_at: str = Field(description="The order creation date")
    order_status: str = Field(description="The current status of the order")
    items: list[OrderItem] = Field(description="The items in the order")

# Set up a parser
parser = PydanticOutputParser(pydantic_object=OrderDetails)

In [30]:
# Define the prompt template
prompt = PromptTemplate(
    template="""
    You are a customer service assistant responsible for processing order changes based on customer input and existing order details.
    
    Given the 'queried_result' which contains existing order details and the 'input' which contains the customer's desired order changes, combine these to finalize the order changes.
    
    Follow these steps:
    1. Review the 'queried_result' to understand the original order details.
    2. Review the 'input' to understand the customer's desired changes.
    3. Combine both sets of information to produce the updated order details.
    
    Ensure that the final output includes only the updated items in the order, formatted the same way as the original order, and set the order_status to '주문 변경'.

    {format_instructions}
    queried_result: {queried_result}
    input: {input}
    """,
    input_variables=["queried_result", "input"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [31]:
# Create the prompt and model pipeline
prompt_and_model = prompt | model

In [32]:
# Example query with sample data
queried_result = {
    "id": 10,
    "created_at": "2024-05-17T12:00:00Z",
    "order_status": "주문 완료",
    "items": [
        {
            "product_name": "떡케익5호",
            "quantity": 1,
            "price": 54000.0
        },
        {
            "product_name": "무지개 백설기 케익",
            "quantity": 1,
            "price": 51500.0
        }
    ]
}

In [33]:
# Invoke the model
output = prompt_and_model.invoke({"queried_result": queried_result, "input": "무지개 백설기 케익만 주문할게요"})
parsed_output = parser.parse(output)

# Print the parsed output
print(parsed_output)


id=10 created_at='2024-05-17T12:00:00Z' order_status='주문 변경' items=[OrderItem(product_name='무지개 백설기 케익', quantity=1, price=51500.0)]


In [34]:
print(output)


    {"id": 10, "created_at": "2024-05-17T12:00:00Z", "order_status": "주문 변경", "items": [{"product_name": "무지개 백설기 케익", "quantity": 1, "price": 51500.0}]}


## products 필드 추가

In [41]:
prompt = PromptTemplate(
    template="""
    You are a customer service assistant responsible for processing order changes based on customer input and existing order details.
    
    Given the 'queried_result' which contains existing order details and the 'input' which contains the customer's desired order changes, combine these to finalize the order changes.
    
    Follow these steps:
    1. Review the 'queried_result' to understand the original order details.
    2. Review the 'input' to understand the customer's desired changes.
    3. Combine both sets of information to produce the updated order details.
    
    Ensure that the final output includes only the updated items in the order, formatted the same way as the original order, and set the order_status to '주문 변경'.

    {format_instructions}
    products: {products}
    queried_result: {queried_result}
    input: {input}
    """,
    input_variables=["products", "queried_result", "input"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [42]:
prompt_and_model = prompt | model

In [43]:
products

[{'product_name': '떡케익5호', 'quantity': 1, 'price': 54000},
 {'product_name': '무지개 백설기 케익', 'quantity': 1, 'price': 51500},
 {'product_name': '미니 백설기', 'quantity': 35, 'price': 31500},
 {'product_name': '개별 모듬팩', 'quantity': 1, 'price': 13500}]

In [48]:
queried_result

{'id': 10,
 'created_at': '2024-05-17T12:00:00Z',
 'order_status': '주문 완료',
 'items': [{'product_name': '떡케익5호', 'quantity': 1, 'price': 54000.0},
  {'product_name': '무지개 백설기 케익', 'quantity': 1, 'price': 51500.0}]}

In [45]:
output = prompt_and_model.invoke({
    "products": products,
    "queried_result": queried_result, 
    "input": "무지개 백설기 케익 빼고 개별 모듬팩 2개 주문할게요"})
parsed_output = parser.parse(output)

In [46]:
print(output)


    {"id": 10, "created_at": "2024-05-17T12:00:00Z", "order_status": "주문 변경", "items": [{"product_name": "떡케익5호", "quantity": 1, "price": 54000.0}, {"product_name": "개별 모듬팩", "quantity": 2, "price": 13500.0}]}


In [47]:
print(parsed_output)

id=10 created_at='2024-05-17T12:00:00Z' order_status='주문 변경' items=[OrderItem(product_name='떡케익5호', quantity=1, price=54000.0), OrderItem(product_name='개별 모듬팩', quantity=2, price=13500.0)]


# 상품 정보 추가하기

In [54]:
from langchain_core.runnables import RunnablePassthrough

RunnablePassthrough.assign(products="파일로 가져온 상품 정보")

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'str'>

In [51]:
order_chagne_prompt = PromptTemplate(
    template="""
    You are a customer service assistant responsible for processing order changes based on customer input and existing order details.
    
    Given the 'queried_result' which contains existing order details and the 'input' which contains the customer's desired order changes, combine these to finalize the order changes.
    
    Follow these steps:
    1. Review the 'queried_result' to understand the original order details.
    2. Review the 'input' to understand the customer's desired changes.
    3. Combine both sets of information to produce the updated order details.
    
    Ensure that the final output includes only the updated items in the order, formatted the same way as the original order, and set the order_status to '주문 변경'.

    {format_instructions}
    products: {products}
    queried_result: {queried_result}
    input: {input}
    """,
    input_variables=["products", "queried_result", "input"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
order_change_prompt

PromptTemplate(input_variables=['input', 'products', 'queried_result'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"id": {"title": "Id", "description": "The order ID", "type": "integer"}, "created_at": {"title": "Created At", "description": "The order creation date", "type": "string"}, "order_status": {"title": "Order Status", "description": "The current status of the order", "type": "string"}, "items": {"title": "Items", "description": "The items in the order", "type": "array", "items": {"$ref": "#/definitions/OrderItem

In [52]:
order_change_chain = order_change_prompt | model
order_change_chain

PromptTemplate(input_variables=['input', 'products', 'queried_result'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"id": {"title": "Id", "description": "The order ID", "type": "integer"}, "created_at": {"title": "Created At", "description": "The order creation date", "type": "string"}, "order_status": {"title": "Order Status", "description": "The current status of the order", "type": "string"}, "items": {"title": "Items", "description": "The items in the order", "type": "array", "items": {"$ref": "#/definitions/OrderItem

In [ ]:
handle_order_change = RunnablePassthrough.assign(products="파일로 가져온 상품 정보") | order_change_chain 
handle_order_change